## Web Scraper

In [1]:
import cloudscraper
from bs4 import BeautifulSoup

Testing on one link before trying to implement a function and apply to all countries of interest

In [2]:
scraper = cloudscraper.create_scraper()  
r = scraper.get("https://fbref.com/en/matches/b9e2f869/Morocco-Brazil-March-25-2023-Friendlies-M")
soup = BeautifulSoup(r.text, "html.parser")
#print(soup.prettify())

In [3]:
table = soup.find('div', attrs = {'id' : 'div_stats_304635c3_summary'}).find('table').find('tbody')
rows = table.find_all('tr')

# Example to display column elements more clearly
ex_row = rows[0]
for col in ex_row:
    ind = ex_row.index(col)
    print('index', ind, ' ' * (2-len(str(ind))), ' | ', col.text, ' ' * (12-len(col.text)), ' | ', col.get('data-stat'))

index 0    |  Rony           |  player
index 1    |  22             |  shirtnumber
index 2    |  FW             |  position
index 3    |  27-318         |  age
index 4    |  63             |  minutes
index 5    |  0              |  goals
index 6    |  0              |  assists
index 7    |  0              |  pens_made
index 8    |  0              |  pens_att
index 9    |                 |  shots
index 10   |                 |  shots_on_target
index 11   |  1              |  cards_yellow
index 12   |  0              |  cards_red
index 13   |                 |  fouls
index 14   |                 |  fouled
index 15   |                 |  offsides
index 16   |                 |  crosses
index 17   |                 |  tackles_won
index 18   |                 |  interceptions
index 19   |  0              |  own_goals
index 20   |                 |  pens_won
index 21   |                 |  pens_conceded


Finding captincy

In [4]:
manager_n_cap = soup.find_all('div', attrs = {'class': 'datapoint'})
cap_name = [manager_n_cap[i].find('a').text for i in [1,3]]

Collect player data for all positions. Save data is stored in a different table. Will pull necessary data from that table if the position is for GK. Otherwise, will just fill the last col data with NA

In [5]:
goalie_table = soup.find('div', attrs = {'id': 'div_keeper_stats_304635c3'}).find('table').find('tbody')

goal_rows = goalie_table.find_all('tr')

brazil_dat = []
for row in rows:
  name_col = row.find_all('th')
  stat_col = row.find_all('td')
  if len(name_col) > 0:
      name = name_col[0].text.strip()

      position = stat_col[1].text
      mins = stat_col[3].text
      goals = stat_col[4].text
      assists = stat_col[5].text
      pens_made = stat_col[6].text
      pens_att = stat_col[7].text
      shots = stat_col[8].text
      shots_on_tar = stat_col[9].text
      yellow_cards = stat_col[10].text
      red_cards = stat_col[11].text
      
      if position == 'GK':
        for r in goal_rows:
          goal_stat = r.find_all('td')
          if len(stat_col) > 0:
            goal_against = goal_stat[3].text
            saves = goal_stat[4].text
            save_per = goal_stat[5].text
      else:
        goal_against = 'NA'
        saves = 'NA'
        save_per = 'NA'
      
      if name in cap_name:
        is_cap = 1
      else:
        is_cap = 0


      
  brazil_dat.append([name,mins,position,goals,assists,pens_made,pens_att,shots,shots_on_tar,yellow_cards,red_cards,goal_against,saves,save_per,is_cap])

In [6]:
brazil_dat[-1]

['Wéverton', '90', 'GK', '0', '0', '0', '0', '', '', '0', '0', '2', '', '', 0]